In [1]:

import pandas as pd
import re
import os

def parse_training_results(file_path):
    """Parse training results from text file into a pandas DataFrame"""
    
    # Read the file
    with open(file_path, 'r') as f:
        content = f.read()
    
    # Initialize lists to store data
    epochs = []
    times = []
    train_losses = []
    train_acc_top1 = []
    train_acc_top5 = []
    test_losses = []
    test_acc_top1 = []
    test_acc_top5 = []
    
    # Regular expression to match each epoch line
    pattern = r'\[Epoch (\d+)\] Time: ([\d.]+)s.*?Train.*?Loss: ([\d.]+).*?Top1: ([\d.]+)%.*?Top5: ([\d.]+)%.*?Test.*?Loss: ([\d.]+).*?Top1: ([\d.]+)%.*?Top5: ([\d.]+)%'
    
    # Find all matches
    matches = re.findall(pattern, content)
    
    for match in matches:
        epochs.append(int(match[0]))
        times.append(float(match[1]))
        train_losses.append(float(match[2]))
        train_acc_top1.append(float(match[3]))
        train_acc_top5.append(float(match[4]))
        test_losses.append(float(match[5]))
        test_acc_top1.append(float(match[6]))
        test_acc_top5.append(float(match[7]))
    
    # Create DataFrame
    df = pd.DataFrame({
        'epoch': epochs,
        'time': times,
        'train_loss': train_losses,
        'train_accuracy_top1': train_acc_top1,
        'train_accuracy_top5': train_acc_top5,
        'test_loss': test_losses,
        'test_accuracy_top1': test_acc_top1,
        'test_accuracy_top5': test_acc_top5
    })
    
    return df


def n_last_average_stats(df, last_n_epochs=5):
    """Calculate average stats over the last N epochs"""
    if len(df) < last_n_epochs:
        last_n_epochs = len(df)
    
    return {
        "ave_train_loss": df['train_loss'].tail(last_n_epochs).mean(),
        "ave_test_loss": df['test_loss'].tail(last_n_epochs).mean(),
        "ave_train_accuracy_top1": df['train_accuracy_top1'].tail(last_n_epochs).mean(),
        "ave_train_accuracy_top5": df['train_accuracy_top5'].tail(last_n_epochs).mean(),
        "ave_test_accuracy_top1": df['test_accuracy_top1'].tail(last_n_epochs).mean(),
        "ave_test_accuracy_top5": df['test_accuracy_top5'].tail(last_n_epochs).mean()
    }

def best_stats(df):
    """Calculate best accuracy achieved during training"""
    return {
        "best_train_accuracy_top1": df['train_accuracy_top1'].max(),
        "best_train_accuracy_top5": df['train_accuracy_top5'].max(),
        "best_test_accuracy_top1": df['test_accuracy_top1'].max(),
        "best_test_accuracy_top5": df['test_accuracy_top5'].max()
    }

def last_epoch_stats(df):
    """Get stats from the last epoch"""
    last_row = df.iloc[-1]
    return {
        "last_epoch_train_loss": last_row['train_loss'],
        "last_epoch_test_loss": last_row['test_loss'],
        "last_epoch_train_accuracy_top1": last_row['train_accuracy_top1'],
        "last_epoch_train_accuracy_top5": last_row['train_accuracy_top5'],
        "last_epoch_test_accuracy_top1": last_row['test_accuracy_top1'],
        "last_epoch_test_accuracy_top5": last_row['test_accuracy_top5']
    }

def parse_params_gflops(file_dir):
    """Parse training results from text file into a pandas DataFrame"""

    train_path = os.path.join(file_dir, "train_eval_results.txt")
    args_path = os.path.join(file_dir, "args.txt")
    
    # Read the file
    with open(train_path, 'r') as f:
        train_content = f.readline()

    with open(args_path, 'r') as f:
        args_content = f.readlines()

    total_params = args_content[-2].strip().split(": ")[1]
    trainable_params = args_content[-1].strip().split(": ")[1]

    gflops = re.search(r'GFLOPs: ([\d.]+)', train_content).group(1)

    return {
        "total_params": int(total_params),
        "trainable_params": int(trainable_params),
        "gflops": float(gflops)
    }



In [2]:
import sys 
import os
import torch 
import torch.nn as nn

curr_dir = os.getcwd()
root_dir = os.path.abspath(os.path.join(curr_dir, '..', '..'))

print("Current Directory:", curr_dir)
print("Root Directory:", root_dir)


Current Directory: /Users/mingikang/Developer/ZiLU-Activation/Output
Root Directory: /Users/mingikang/Developer


In [3]:
save_path =  os.path.join(curr_dir, "csv")
print("Save Path:", save_path)
print(curr_dir)

results_path = os.path.join(curr_dir, "AUG/LR-ResNet34/") # Change for ResNet / ViT
print("Results Path:", results_path)

Save Path: /Users/mingikang/Developer/ZiLU-Activation/Output/csv
/Users/mingikang/Developer/ZiLU-Activation/Output
Results Path: /Users/mingikang/Developer/ZiLU-Activation/Output/AUG/LR-ResNet34/


In [12]:
# 
datasets = ["CIFAR10", "CIFAR100"]
lrs = ["1e-2", "1e-3", "1e-4", "1e-5"]
# activations = ["zilu", "zilu_approx"]
base_activations = ["relu", "gelu", "silu"]
activations = ["zilu", "zilu_approx"]
sigmas = ["0.1", "0.5", "1.0", "5.0", "10.0"]


rows = [] 
for ds in datasets: 

    save_path_train = os.path.join(save_path, f"LR-ResNet34/{ds}_activations_train_loss.csv")
    save_path_test = os.path.join(save_path, f"LR-ResNet34/{ds}_activations_test_loss.csv")
    print(f"Saving to {save_path_train} and {save_path_test}")

    # Base Activations 
    act_test_loss = {}
    act_train_loss = {}
    
    for lr in lrs:
        for ba in base_activations:
            dir_path = os.path.join(results_path, ds, f"{ba}_lr{lr}_s42")
            txt_path = os.path.join(dir_path, "train_eval_results.txt")

            print(f"Processing {txt_path} and saving to {save_path_train} and {save_path_test}")

            df = parse_training_results(txt_path)

            test_loss = df["test_loss"].tolist() 
            train_loss = df["train_loss"].tolist()


            act_test_loss[f"{ba}_{lr}_test_loss"] = test_loss
            act_train_loss[f"{ba}_{lr}_train_loss"] = train_loss
        

    # ZiLU Activations    
    for lr in lrs:
        for act in activations:
            for sigma in sigmas:
                dir_path = os.path.join(results_path, ds, f"{act}_sigma{sigma}_lr{lr}_s42")
                txt_path = os.path.join(dir_path, "train_eval_results.txt")

                print(f"Processing {txt_path} and saving to {save_path_train} and {save_path_test}")

                df = parse_training_results(txt_path)

                test_loss = df["test_loss"].tolist() 
                train_loss = df["train_loss"].tolist()


                act_test_loss[f"{act}_sigma{sigma}_lr{lr}_test_loss"] = test_loss
                act_train_loss[f"{act}_sigma{sigma}_lr{lr}_train_loss"] = train_loss
        
    df_train = pd.DataFrame(act_train_loss)
    df_train.insert(0, "epoch", list(range(1, len(df_train)+1)))
    df_train.to_csv(save_path_train, index=False)

    df_test = pd.DataFrame(act_test_loss)
    df_test.insert(0, "epoch", list(range(1, len(df_test)+1)))
    df_test.to_csv(save_path_test, index=False)

    


Saving to /Users/mingikang/Developer/ZiLU-Activation/Output/csv/LR-ResNet34/CIFAR10_activations_train_loss.csv and /Users/mingikang/Developer/ZiLU-Activation/Output/csv/LR-ResNet34/CIFAR10_activations_test_loss.csv
Processing /Users/mingikang/Developer/ZiLU-Activation/Output/AUG/LR-ResNet34/CIFAR10/relu_lr1e-2_s42/train_eval_results.txt and saving to /Users/mingikang/Developer/ZiLU-Activation/Output/csv/LR-ResNet34/CIFAR10_activations_train_loss.csv and /Users/mingikang/Developer/ZiLU-Activation/Output/csv/LR-ResNet34/CIFAR10_activations_test_loss.csv
Processing /Users/mingikang/Developer/ZiLU-Activation/Output/AUG/LR-ResNet34/CIFAR10/gelu_lr1e-2_s42/train_eval_results.txt and saving to /Users/mingikang/Developer/ZiLU-Activation/Output/csv/LR-ResNet34/CIFAR10_activations_train_loss.csv and /Users/mingikang/Developer/ZiLU-Activation/Output/csv/LR-ResNet34/CIFAR10_activations_test_loss.csv
Processing /Users/mingikang/Developer/ZiLU-Activation/Output/AUG/LR-ResNet34/CIFAR10/silu_lr1e-2_s4

In [13]:
df_test

,epoch,relu_1e-2_test_loss,gelu_1e-2_test_loss,silu_1e-2_test_loss,relu_1e-3_test_loss,gelu_1e-3_test_loss,silu_1e-3_test_loss,relu_1e-4_test_loss,gelu_1e-4_test_loss,silu_1e-4_test_loss,...,zilu_sigma0.1_lr1e-5_test_loss,zilu_sigma0.5_lr1e-5_test_loss,zilu_sigma1.0_lr1e-5_test_loss,zilu_sigma5.0_lr1e-5_test_loss,zilu_sigma10.0_lr1e-5_test_loss,zilu_approx_sigma0.1_lr1e-5_test_loss,zilu_approx_sigma0.5_lr1e-5_test_loss,zilu_approx_sigma1.0_lr1e-5_test_loss,zilu_approx_sigma5.0_lr1e-5_test_loss,zilu_approx_sigma10.0_lr1e-5_test_loss
0,1,4.168974,4.217269,4.222132,4.522007,4.203504,4.150528,3.635032,3.729327,3.524018,...,3.586317,4.473214,4.282167,4.261740,4.278876,3.592422,4.312796,4.268754,4.332016,4.228973
1,2,3.752686,3.906902,4.082196,3.278353,3.425000,3.863820,3.249264,3.207518,3.164153,...,3.329679,3.869777,3.850179,3.941484,3.970509,3.326622,3.730198,3.805854,3.928312,3.923698
2,3,3.682812,3.604119,3.839162,3.132056,3.112892,3.590951,3.063069,2.992111,2.925140,...,3.173047,3.566498,3.624149,3.746693,3.778063,3.165231,3.487413,3.575710,3.724657,3.733312
3,4,3.338981,3.315143,3.643106,3.357501,2.798382,3.326628,2.836369,2.834809,2.769813,...,3.056481,3.395815,3.465399,3.628608,3.657767,3.033434,3.348088,3.437923,3.587424,3.614767
4,5,3.201828,4.805721,3.530839,3.358408,2.617897,3.025977,2.795538,2.713479,2.667898,...,2.977609,3.263794,3.359696,3.538224,3.551334,2.959984,3.241102,3.335887,3.488702,3.529862
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,196,1.900894,2.876567,2.643648,3.347163,3.567526,3.490783,3.488562,3.680210,3.539308,...,1.972415,3.424942,3.544696,3.337467,3.227797,2.158019,3.378958,3.500090,3.355013,3.296079
196,197,1.867569,2.944894,2.823273,3.389776,3.553614,3.459352,3.479317,3.690183,3.568513,...,1.980157,3.417434,3.560330,3.366371,3.279944,2.151550,3.391297,3.510895,3.333835,3.291944
197,198,2.038426,2.866042,2.714477,3.419371,3.534347,3.460225,3.441161,3.720028,3.573001,...,1.984477,3.410684,3.554546,3.360526,3.259033,2.161745,3.416902,3.503953,3.359028,3.313660
198,199,2.166223,2.866263,2.736199,3.407188,3.532325,3.459929,3.470325,3.661497,3.561840,...,2.006872,3.408650,3.542681,3.348255,3.266639,2.177969,3.419430,3.525435,3.355384,3.305506
